In [1]:
import ast
import pandas as pd

In [2]:
data = pd.read_csv("data/processed.csv").reset_index(drop=True)

## Create features

In [4]:
def count_sentences(dist):
    return len(eval(dist))

In [5]:
#remove records with too many sentences
data["no_of_sentences"] = data["eucl_dis"].apply(count_sentences)
data = data[data["no_of_sentences"] < 13]
del data["no_of_sentences"]
data = data.reset_index(drop=True)

In [8]:
features = pd.DataFrame()
     
for i in range(len(data["eucl_dis"])):
    eucl_dis = eval(data["eucl_dis"][i])
    for j in range(12):
        if j < len(eucl_dis):
            features.loc[i, "eucl_dis_" + str(j)] = eucl_dis[j]
        else: 
            features.loc[i, "eucl_dis_" + str(j)] = 10

In [9]:
for i in range(len(data["cosine_dis"])):
    cosine_dis = eval(data["cosine_dis"][i].replace("nan","1"))
    for j in range(12):
        if j < len(cosine_dis):
            features.loc[i, "cosine_dis_" + str(j)] = cosine_dis[j]
        else:
            features.loc[i, "cosine_dis_" + str(j)] = 1

In [10]:
features["ans_sentence"] = data["ans_sentence"]

In [11]:
features

,eucl_dis_0,eucl_dis_1,eucl_dis_2,eucl_dis_3,eucl_dis_4,eucl_dis_5,eucl_dis_6,eucl_dis_7,eucl_dis_8,eucl_dis_9,...,cosine_dis_3,cosine_dis_4,cosine_dis_5,cosine_dis_6,cosine_dis_7,cosine_dis_8,cosine_dis_9,cosine_dis_10,cosine_dis_11,ans_sentence
0,4.877357,4.781120,3.989753,5.392237,10.0,10.0,10.0,10.0,10.0,10.0,...,0.450240,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,4.541980,4.213068,3.648401,4.946164,10.0,10.0,10.0,10.0,10.0,10.0,...,0.354981,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
2,4.032178,3.851763,3.576980,4.522441,10.0,10.0,10.0,10.0,10.0,10.0,...,0.282117,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3
3,4.703169,4.480725,3.853026,5.213559,10.0,10.0,10.0,10.0,10.0,10.0,...,0.410191,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
4,4.480060,4.477153,3.738475,5.006326,10.0,10.0,10.0,10.0,10.0,10.0,...,0.364631,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85882,3.404869,3.019961,3.916248,3.837237,10.0,10.0,10.0,10.0,10.0,10.0,...,0.285989,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
85883,4.280803,4.052094,4.874015,4.917758,10.0,10.0,10.0,10.0,10.0,10.0,...,0.548615,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
85884,3.665919,3.494764,4.085484,3.989559,10.0,10.0,10.0,10.0,10.0,10.0,...,0.313942,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2
85885,3.471442,3.317085,3.995896,3.755651,10.0,10.0,10.0,10.0,10.0,10.0,...,0.271118,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1


## Split data and run models

In [43]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features.iloc[:,:-1])

In [44]:
train_feat, test_feat, train_ans, test_ans = train_test_split(features_scaled, features.iloc[:,-1])

In [45]:
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [46]:
regression = linear_model.LogisticRegression(solver='saga', multi_class='multinomial')
regression.fit(train_feat, train_ans)

regression_train_predictions = regression.predict(train_feat)
regression_test_predictions = regression.predict(test_feat)

regression_train_acc = metrics.accuracy_score(train_ans, regression_train_predictions)
regression_test_acc = metrics.accuracy_score(test_ans, regression_test_predictions)

print("Logistic regression results:\ntrain set: " + str(regression_train_acc) + "\ntest set: " + str(regression_test_acc))

Logistic regression results:
train set: 0.6405495614375534
test set: 0.6475409836065574


In [47]:
forest = RandomForestClassifier(criterion='entropy')
forest.fit(train_feat, train_ans)

forest_train_predictions = forest.predict(train_feat)
forest_test_predictions = forest.predict(test_feat)

forest_train_acc = metrics.accuracy_score(train_ans, forest_train_predictions)
forest_test_acc = metrics.accuracy_score(test_ans, forest_test_predictions)

print("Random forest results:\ntrain set: " + str(forest_train_acc) + "\ntest set: " + str(forest_test_acc))

Random forest results:
train set: 0.9997360863152992
test set: 0.6873137108792846


In [51]:
xg = xgb.XGBClassifier(max_depth = 5)
xg.fit(train_feat, train_ans)
xg_train_predictions = xg.predict(train_feat)
xg_test_predictions = xg.predict(test_feat)

xg_train_acc = metrics.accuracy_score(train_ans, xg_train_predictions)
xg_test_acc = metrics.accuracy_score(test_ans, xg_test_predictions)

print("XGBoost results:\ntrain set: " + str(xg_train_acc) + "\ntest set: " + str(xg_test_acc))

XGBoost results:
train set: 0.7748971512846387
test set: 0.7099944113263785
